In [ ]:
!pip install tensorflow

In [1]:
#import libraries
import pandas as pd
import jax
import jax.numpy as jnp

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from collections import namedtuple

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
test_data = pd.read_csv("train.csv")
train_data = pd.read_csv("train.csv")

pclasses = jnp.array(train_data['Pclass']).reshape((-1, 1))
survived = jnp.array(train_data['Survived']).reshape((-1, 1))

In [5]:
# We need 2 parameter: weights (w) and bias (b) for the regrssion

LogisticRegressionParams = namedtuple('LogisticRegressionParams', 'w b')

In [6]:
#logistic function

def sigmoid_predict(params: LogisticRegressionParams, x: jnp.array) -> jnp.array:
    z = x.dot(params.w) + params.b
    return jax.nn.sigmoid(z)

In [7]:
#softmax function

@jax.jit
def softmax_predict(params: LogisticRegressionParams, x: jnp.array) -> jnp.array:
    z = params.w.transpose() @ x + params.b
    return jax.nn.softmax(z)

In [8]:
# vectorize the functions

sigmoid_vpredict = jax.vmap(sigmoid_predict,(None, 0))
softmax_vpredict = jax.vmap(softmax_predict,(None, 0))

In [9]:
@jax.jit
def crossent(params: jnp.array, features: jnp.array, labels: jnp.array) -> jnp.array:
    predictions = sigmoid_vpredict(params, features)
    a = labels * jnp.log(predictions)
    b = (1.0 - labels) * jnp.log(1.0 - predictions)
    return -jnp.mean(a + b)

In [10]:
learning_rate = 1e-2
sigmoid_params = LogisticRegressionParams(jnp.array(1.0), jnp.array(1.0))
crossent_grad_fn = jax.grad(crossent)

for i in range(1_000):
    if i % 100 == 0:
        print(crossent(sigmoid_params, pclasses, survived))
    grads = crossent_grad_fn(sigmoid_params, pclasses, survived)
    sigmoid_params = LogisticRegressionParams(sigmoid_params.w - learning_rate * grads.w, sigmoid_params.b - learning_rate * grads.b)

2.227012
0.7300361
0.62537605
0.62087697
0.620122
0.6195704
0.61905324
0.6185613
0.6180927
0.6176463


In [11]:
# see how it  performs
preds = round(sigmoid_vpredict(sigmoid_params, pclasses))
accuracy = 1.0 - abs(preds-survived).mean()
print(f"accuracy is {100.0*accuracy:.2f}%")

accuracy is 67.90%
